In [2]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [79]:
from src.utils import etl
import importlib
importlib.reload(etl)
scraper = etl.ReviewScraper()

Driver Got


In [63]:
def get_driver(debug=False):
    options = Options()

    if not debug:
        options.add_argument("--headless")
        options.add_argument('--no-sandbox')
    else:
        options.add_argument("--window-size=1366,768")
        

    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    driv_path = '../config/webdrivers/chromedriver.exe'
    input_driver = webdriver.Chrome(options=options, executable_path=driv_path)
    return input_driver

In [44]:
map_url

'https://maps.google.com/maps?q=Foodies+Cafe+436+South+Parramore+Avenue+b,+Orlando&um=1&ie=UTF-8&sa=X&ved=2ahUKEwip3J-XsLTwAhVTOs0KHSI-C5QQ_AUoAXoECAEQAw'

In [16]:
file_list.index(file)

247

In [80]:
dpath = '../data/outputs/'+'cbg_restaurants/recollect/'

file_list = [dpath+ f for f in os.listdir(dpath)]
for file in file_list[251:]:
    df = pd.read_json(file)
    for idx,i in df.iterrows():
        cbg, rest_name,rest_add,place_id = i['CBG'],i['name'],i['vicinity'],i['reference']
        rev_dpath = '../data/outputs/reviews/'
        
        fpath = rev_dpath + str(cbg)+ '_' + place_id + '.json'
        rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)]
        if 'Hotel' in rest_name or 'InterContinental' in rest_name or 'Plaza' in rest_name \
                or 'Inn' in rest_name or 'Leku' in rest_name or 'American Girl' in rest_name \
                or 'Courtyard' in rest_name or 'Hilton' in rest_name or 'Graduate Berkeley' in rest_name \
                or 'Campton' in rest_name or 'Bagatelle' in rest_name or 'Tudor House' in rest_name \
                or 'Hyatt' in rest_name:
            continue
        
        if fpath not in rev_files:
            url = 'https://www.google.com/search?q=' + rest_name + ' ' + rest_add
            driver = get_driver()
    
            driver.get(url)
            response = BeautifulSoup(driver.page_source, 'html.parser')

            for res in response.find_all('a', href=True):
                if 'maps.google' in res['href']:
                    map_url  = res['href']
            place_data = scraper.get_place_data(map_url)
            n_reviews = place_data['n_reviews']
            print(n_reviews)
#             if n_reviews > 50:
#                 continue
            if n_reviews >0:
                if n_reviews > 500:
                    n_reviews = 0.8 * n_reviews
                revs = scraper.get_reviews(n_reviews,map_url,cbg,place_id)

    

127
Reviews Sorted
636


TypeError: 'NoneType' object is not subscriptable